In [1]:
1+1

2

In [7]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain import hub

# 환경 변수
load_dotenv()
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")


# 기존 도구 함수들
def add_two(x: str) -> str:
    try:
        return str(float(x) + 2)
    except Exception:
        raise ValueError("숫자만 입력해 주세요.")


def population_lookup(city: str) -> str:
    data = {
        "서울": "9,733,509명 (2025년 기준)",
        "부산": "3,400,000명 (2025년 기준)",
    }
    result = data.get(city.strip().lower())
    if result:
        return result
    else:
        raise ValueError(f"{city}의 인구 정보를 찾을 수 없습니다.")


# 도구 및 LLM 설정
tools = [
    Tool(name="AddTwo", func=add_two, description="숫자 입력 시 2를 더합니다."),
    Tool(
        name="PopulationLookup",
        func=population_lookup,
        description="도시 이름 입력 시 인구 정보를 반환합니다.",
    ),
]

llm = ChatOpenAI(
    model="gpt-4o-mini", temperature=0, openai_api_key=OPEN_API_KEY, verbose=True
)

# ReAct 프롬프트 설정 (방법 1: 직접 정의)
react_prompt = PromptTemplate.from_template(
    """
당신은 도움이 되는 어시스턴트입니다. 주어진 도구를 사용해서 질문에 답하세요.

사용 가능한 도구:
{tools}

도구 이름: {tool_names}

다음 형식을 사용하세요:

Question: 답변해야 할 입력 질문
Thought: 무엇을 해야 할지 생각해보세요
Action: 수행할 액션, [{tool_names}] 중 하나여야 합니다
Action Input: 액션에 대한 입력
Observation: 액션의 결과
... (이 Thought/Action/Action Input/Observation을 필요한 만큼 반복)
Thought: 이제 최종 답변을 알았습니다
Final Answer: 원래 입력 질문에 대한 최종 답변

시작하세요!

Question: {input}
Thought: {agent_scratchpad}
"""
)

# 또는 방법 2: LangChain Hub에서 기본 ReAct 프롬프트 사용
# react_prompt = hub.pull("hwchase17/react")

# Agent 생성
# agent = create_react_agent(llm, tools, react_prompt, debug=True)
agent = create_react_agent(llm=llm, tools=tools, prompt=react_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


# Fallback 함수
def llm_fallback(inputs):
    user_input = inputs.get("input", "")
    response = llm.invoke(f"다음 질문에 답해주세요: {user_input}")
    return {
        "output": f"🤖 도구에서 정보를 찾지 못해 일반 지식으로 답변합니다:\n{response.content}",
        "intermediate_steps": [],
    }


# Fallback이 적용된 Agent
smart_agent = agent_executor.with_fallbacks([RunnableLambda(llm_fallback)])

# 테스트
if __name__ == "__main__":
    # 도구에서 찾을 수 있는 정보
    print("=== 도구에서 찾을 수 있는 정보 ===")
    result1 = smart_agent.invoke({"input": "서울의 인구는?"})
    print(result1["output"])

    print("\n=== 도구에서 찾을 수 없는 정보 (LLM Fallback) ===")
    result2 = smart_agent.invoke({"input": "대구의 인구는?"})
    print(result2["output"])

=== 도구에서 찾을 수 있는 정보 ===


> Entering new AgentExecutor chain...
서울의 인구 정보를 알아보아야 합니다. 
Action: PopulationLookup 
Action Input: "서울" 9,733,509명 (2025년 기준)Thought: 서울의 인구는 9,733,509명입니다. 이제 최종 답변을 정리하겠습니다.
Final Answer: 서울의 인구는 9,733,509명입니다.

> Finished chain.
서울의 인구는 9,733,509명입니다.

=== 도구에서 찾을 수 없는 정보 (LLM Fallback) ===


> Entering new AgentExecutor chain...
대구의 인구 정보를 알아보아야 합니다. 
Action: PopulationLookup 
Action Input: "대구" 🤖 도구에서 정보를 찾지 못해 일반 지식으로 답변합니다:
대구의 인구는 2023년 기준으로 약 240만 명 정도입니다. 하지만 인구는 시간이 지남에 따라 변동할 수 있으므로, 최신 정보를 확인하는 것이 좋습니다. 대구에 대한 더 구체적인 정보가 필요하시면 말씀해 주세요!
